In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare

In [ ]:
# Unzip the dataset
!unzip /content/drive/MyDrive/Grad/AffectNet-8Labels.zip -d /content/data

# Extract the tar files
!tar -xf "/content/data/train_set.tar" -C "/content/"
!tar -xf "/content/data/val_set.tar" -C "/content/"

In [ ]:
# Import Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# Define the paths
train_path = "/content/train_set"
val_path = "/content/val_set"

In [ ]:
train_set = os.listdir(f"{train_path}/images")
val_set = os.listdir(f"{val_path}/images")

# count the number of images in the train_set
train_file_count = 0
for f in train_set:
  if f.endswith(".jpg"):
    train_file_count += 1

# count the number of images in the val_set
val_file_count = 0
for f in val_set:
  if f.endswith(".jpg"):
    val_file_count += 1

train_file_count, val_file_count, train_file_count + val_file_count

(287651, 3999, 291650)

In [ ]:
# Function to extract annotations from numpy files
def extract_annotations(image_id, annotations_path):
    annotations = {}
    for field in ['aro', 'exp', 'val']:
        file_path = os.path.join(annotations_path, f'{image_id}_{field}.npy')
        if os.path.exists(file_path):
            with open(file_path, 'rb') as file:
                if field == 'exp':
                  annotations[field] = np.load(file).astype(int)
                else:
                  annotations[field] = np.load(file).astype(np.float64)
        else:
          print(f"File path couldn't find. Skipping file {image_id}_{field}.npy")
          print(f"File path: {file_path}")
    return annotations

In [ ]:
def build_annotations(img_path, annotations_path, annotations_csv_path):

  # Define the list to store annotations
  annotations_list = []

  for img in os.listdir(img_path):
    # Get image ID from filename
    id = img.split(".")[0]

    # Extract annotations
    annotations = extract_annotations(id, annotations_path)

    # Append to annotations list
    annotations_list.append({
                'id': id,
                'arousal': annotations['aro'],
                'valance': annotations['val'],
                'expression': annotations['exp'],
            })

  # Create DataFrame from the annotations list
  annotations_df = pd.DataFrame(annotations_list)
  # Save DataFrame to CSV file
  annotations_df.to_csv(annotations_csv_path, index=False)

  print(f'Annotations CSV file saved at: {annotations_csv_path}')
  return annotations_df

In [ ]:
# Paths for train set
train_images = "/content/train_set/images"
train_annotations = "/content/train_set/annotations"
train_csv = "/content/drive/MyDrive/Grad/dataset/train_annotations.csv"

# Paths for val set
val_images = "/content/val_set/images"
val_annotations = "/content/val_set/annotations"
val_csv = "/content/drive/MyDrive/Grad/dataset/val_annotations.csv"

train_df = build_annotations(train_images, train_annotations, train_csv)
val_df = build_annotations(val_images, val_annotations, val_csv)

Annotations CSV file saved at: /content/drive/MyDrive/Grad/dataset/train_annotations.csv
Annotations CSV file saved at: /content/drive/MyDrive/Grad/dataset/val_annotations.csv


In [ ]:
!tar -cvf "/content/drive/MyDrive/Grad/dataset/train_images.tar" "/content/train_set/images"
!tar -cvf "/content/drive/MyDrive/Grad/dataset/val_images.tar" "/content/val_set/images"

# Split

In [2]:
test = "/content/drive/MyDrive/Graduation Project/AffectNet/test_annotations.csv"
train = "/content/drive/MyDrive/Graduation Project/AffectNet/train_annotations.csv"
val = "/content/drive/MyDrive/Graduation Project/AffectNet/val_annotations.csv"

In [3]:
import pandas as pd

train_df = pd.read_csv(train)
val_df = pd.read_csv(val)
test_df = pd.read_csv(test)

In [4]:
train_df.shape, val_df.shape, test_df.shape

((283651, 4), (3999, 4), (4000, 4))

In [14]:
train_df.head()

,id,arousal,valance,expression
0,411701,-0.101626,0.827526,1
1,167867,0.690476,-0.119048,3
2,325007,0.375259,-0.138026,6
3,382631,0.313808,0.772066,1
4,195343,0.183150,-0.408566,0
